This notebook demonstrate how to use ifcopenshell to extract materials used in walls of a building model.

First we need to install required packages.

In [1]:
%pip install ifcopenshell
%pip install lark numpy ## some package missing in the ifcopenshell
%pip install

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import ifcopenshell and open the IFC file.

In [3]:
import os
import ifcopenshell
import pandas as pd

In [4]:
file_path = os.path.join(os.getcwd(), '..', 'examples', 'basic-house.ifc')
model = ifcopenshell.open(file_path)

Now we have the IFC model loaded. We are going to extract the materials used in the walls of the building. Including the common wall properties such as is external, is load bearing. We will also extract the quantity and dimensions of the wall materials.

In [10]:
# Extract walls
def extract_walls(ifc_file):
    """Extract wall elements from the IFC file."""
    return ifc_file.by_type('IfcWall')

extracted_walls = extract_walls(model)
print(f'Number of walls: {len(extracted_walls)}')


Number of walls: 13


#361=IfcWallStandardCase('2DedXznHnDaeAWsrTB_qBp',#41,'Basic Wall:Yttervägg Paroc:1298028',$,'Basic Wall:Yttervägg Paroc',#311,#359,'1298028')

In [73]:
extracted_walls = extract_walls(model)
print(f'Number of walls: {len(extracted_walls)}')
extracted_walls[0]

Number of walls: 13


#361=IfcWallStandardCase('2DedXznHnDaeAWsrTB_qBp',#41,'Basic Wall:Yttervägg Paroc:1298028',$,'Basic Wall:Yttervägg Paroc',#311,#359,'1298028')

In [79]:
def format_material_name(material):
    """Format the material name to a readable format."""
    if material and material.is_a('IfcMaterial'):
        return material.Name
    return str(material)

def extract_material(wall):
    """Extract material information for a given wall, considering material layers if present."""
    material_info = []
    material_association = wall.HasAssociations
    if material_association:
        for assoc in material_association:
            if assoc.is_a('IfcRelAssociatesMaterial'):
                material = assoc.RelatingMaterial
                if material.is_a('IfcMaterialLayerSetUsage'):
                    for layer in material.ForLayerSet.MaterialLayers:
                        material_info.append({
                            'Material': format_material_name(layer.Material),
                            'LayerThickness': layer.LayerThickness
                        })
                elif material.is_a('IfcMaterialLayerSet'):
                    for layer in material.MaterialLayers:
                        material_info.append({
                            'Material': format_material_name(layer.Material),
                            'LayerThickness': layer.LayerThickness
                        })
                else:
                    material_info.append({
                        'Material': format_material_name(material),
                        'LayerThickness': None
                    })
                break
    return material_info

In [80]:
wall = extracted_walls[0]
material_info = extract_material(wall)
print(material_info)

[{'Material': 'Betong', 'LayerThickness': 540.0}]


In [75]:
def extract_pset_wall_common(wall):
    """Extract PsetWallCommon properties for a given wall."""
    psets = ifcopenshell.util.element.get_psets(wall)
    pset_wall_common = psets.get('Pset_WallCommon', {})
    return pset_wall_common


In [76]:
wall_common_properties = extract_pset_wall_common(wall)
print(wall_common_properties)

{'Reference': 'Yttervägg Paroc', 'IsExternal': True, 'id': 530, 'ExtendToStructure': True, 'LoadBearing': False}


In [77]:
def extract_base_quantities(wall):
    """Extract base quantities for a given wall."""
    psets = ifcopenshell.util.element.get_psets(wall)
    pset_base_quantities = psets.get('BaseQuantities', {})
    return pset_base_quantities

In [78]:
wall_base_quantities = extract_base_quantities(wall)
print(wall_base_quantities)

{'Height': 2534.63712153532, 'Length': 22600.0, 'Width': 540.0, 'GrossFootprintArea': 12.204, 'NetVolume': 25.1098377156085, 'NetSideArea': 48.0210989466982, 'id': 378}


In [81]:
def create_material_passport(model):
    """Create a material passport for all wall elements in the IFC file."""
    walls = extract_walls(model)
    material_passport = []

    for wall in walls:
        wall_data = {'WallID': wall.GlobalId}

        # Extract PsetWallCommon properties and add to wall_data
        pset_wall_common = extract_pset_wall_common(wall)
        wall_data.update(pset_wall_common)

        # Extract base quantities (assuming part of Pset_PackingInstructions) and add to wall_data
        base_quantities = extract_base_quantities(wall,)
        wall_data['BaseQuantities'] = base_quantities

        # Extract material and add to wall_data
        materials_info = extract_material(wall)
        for material_info in materials_info:
            material_passport.append({
                'WallID': wall.GlobalId,
                'Material': material_info['Material'],
                'LayerThickness': material_info['LayerThickness'],
                **pset_wall_common,
                **base_quantities
            })

    return pd.DataFrame(material_passport)

In [83]:
material_passport = create_material_passport(model)
material_passport.head()

,WallID,Material,LayerThickness,Reference,IsExternal,id,ExtendToStructure,LoadBearing,Height,Length,Width,GrossFootprintArea,NetVolume,NetSideArea
0,2DedXznHnDaeAWsrTB_qBp,Betong,540.0,Yttervägg Paroc,True,378,True,False,2534.637122,22600.0,540.0,12.2040,25.109838,48.021099
1,2DedXznHnDaeAWsrTB_qBo,Betong,540.0,Yttervägg Paroc,True,645,True,False,3584.308014,8960.0,540.0,4.8384,12.016680,22.253111
2,2DedXznHnDaeAWsrTB_qBn,Betong,540.0,Yttervägg Paroc,True,794,True,False,2534.637122,22060.0,540.0,11.9124,16.853897,32.682895
3,2DedXznHnDaeAWsrTB_qBm,Betong,540.0,Yttervägg Paroc,True,952,True,False,3584.308014,8420.0,540.0,4.5468,11.167448,20.680459
4,2DedXznHnDaeAWsrTB_qBj,Betong,170.0,Innervägg 170mm Paroc,False,1072,False,False,2400.000000,8420.0,170.0,1.4314,2.782526,16.367800


In [84]:
## Save the material passport to a CSV file
material_passport.to_csv('material_passport.csv', index=False)